In [5]:
import matplotlib.pyplot as plt
from tbparse import SummaryReader

In [7]:
data = SummaryReader("./yahboom_obstacle_avoidance/logs/events.out.tfevents.1762906612.directlab-System-Product-Name")

TypeError: Cannot convert numpy.ndarray to numpy.ndarray